# 3.1 일반적인 매핑형

`collections.abc` 모듈은 `dict` 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 `Mapping` 및 `MutableMapping` 추상 베이스 클래스를 제공한다.

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

표준 라이브러리에서 제공하는 매핑형은 모두 `dict`를 이용해서 구현하므로, 키가 __해시 가능__해야한다. 여기서 해시 가능하다는 말은 변하지 않는 해시값을 갖고 있고(`__hash__()` 메서드) 다른 객체와 비교할 수 있다(`__eq__()` 메서드)는 말이다. 원자적 불변형(str, byte, 수치형)은 모두 해시 가능하며, frozenset 또한 해시 가능하다. 튜플은 모든 항목들이 모두 해시 가능해야 해시 가능하다.

In [2]:
tt = (1,2,(30,40))
hash(tt)

8027212646858338501

In [3]:
tl = (1,2,[30,40])
hash(tl)

TypeError: unhashable type: 'list'

In [4]:
tf = (1,2,frozenset([30,40]))
hash(tf)

985328935373711578

딕셔너리는 다음과 같이 다양한 방법으로 구현할 수 있다.

In [5]:
a = dict(one=1, two=2)
a

{'one': 1, 'two': 2}

In [7]:
b = {'one':1, 'two':2}
b

{'one': 1, 'two': 2}

In [8]:
c = dict(zip(['one','two'],[1,2]))
c

{'one': 1, 'two': 2}

In [11]:
d = dict([('two',2),('one',1)])
d

{'two': 2, 'one': 1}

In [12]:
e = dict({'one':1, 'two':2})
e

{'one': 1, 'two': 2}

In [13]:
a == b == c == d == e

True

# 3.2 지능형 딕셔너리

In [15]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (55, 'Brazil'),
]

In [16]:
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'Brazil': 55}

In [21]:
{code: country.upper() for country, code in country_code.items()
    if code < 70}

{55: 'BRAZIL'}

# 3.3 공통적인 매핑 메서드

`d.update(m, [**kargs])` 메서드는 (키, 값) 쌍의 매핑이나 반복형 객체에서 가져온 항목들로 d를 갱신한다. 이 메서드가 첫 번째 인수 m을 다루는 방식은 덕 타이핑의 대표적인 사례이다.

1. 먼저 m이 `keys()` 메서드를 갖고 있으면 매핑이라고 간주한다.
1. 메서드가 없으면 `update()` 메서드는 m의 항목들이 (키, 값) 쌍으로 되어 있다고 간주하고 m을 반복한다.

## 3.3.1 존재하지 않는 키를 setdefault()로 처리하기

존재하지 않는 키 k로 `d[k]`를 접근하면 조기 실패 철학에 따라 `dict`는 오류를 발생시킨다. 이 경우 `d[k]` 대신 `d.get(k, default)`를 사용하면 대신 `default`를 반환하도록 할 수 있지만, 이는 효율성이 좋지 않다.

In [29]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
def index0(txt_file):
    import sys, re

    WORD_RE = re.compile(r'\w+')
    index = {}

    with open(txt_file, encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                occurence = index.get(word, [])
                occurence.append(location)
                index[word] = occurence
                
    for word in sorted(index, key=str.upper):
        print(word, index[word])

In [28]:
index0('zen.txt')

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

위의 코드를 `dict.setdefault()`를 사용하여 한 줄로 수정할 수 있다.

> `dict.setdefault()` : k in d가 참이면 `d[k]`를 반환하고, 아니면 `d[k]=default`로 설정하고 이 값을 반환한다.

In [32]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
def index1(txt_file):
    import sys, re

    WORD_RE = re.compile(r'\w+')
    index = {}

    with open(txt_file, encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                index.setdefault(word, []).append(location)
                
    for word in sorted(index, key=str.upper):
        print(word, index[word])

In [33]:
index1('zen.txt')

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

# 3.4 융퉁성 있게 키를 조회하는 매핑

검색할 때 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리하다. 이는 

1. `dict` 대신 `defaultdict`를 쓰거나, 

1. `dict` 등의 매핑형을 상속해서 `__missing__()` 메서드를 추가하면 된다.

## 3.4.1 `defaultdict`: 존재하지 않는 키에 대한 또 다른 처리

`defaultdict`는 존재하지 않는 키로 검색할 때 요청에 따라 항목을 생성한다. 작동 방식은 존재하지 않는 키 인수로 `__getitem__()` 메서드를 호출할 때마다 기본값을 생성하기 위해 사용되는 콜러블을 제공하는 것이다.

예를 들어 `dd = defaultdict(list)` 코드로 생성한 후 존재하지 않는 키인 'new-key'로 `dd['new-key']` 표현식을 실행하면 다음과 같이 처리된다.

1. 리스트를 새로 생성하기 위해 `list()`를 호출한다.
1. 'new-key'를 키로 사용해서 새로운 리스트를 dd에 삽입한다.
1. 리스트에 대한 참조를 반환한다.

In [34]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
def index2(txt_file):
    import sys, re, collections

    WORD_RE = re.compile(r'\w+')
    index = collections.defaultdict(list)

    with open(txt_file, encoding='utf-8') as fp:
        for line_no, line in enumerate(fp, 1):
            for match in WORD_RE.finditer(line):
                word = match.group()
                column_no = match.start()+1
                location = (line_no, column_no)
                index[word].append(location)
                
    for word in sorted(index, key=str.upper):
        print(word, index[word])

In [35]:
index2('zen.txt')

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

다만 `defaultdict`는 `__getitem__()` 호출에 대한 기본값을 제공할 뿐, 다른 메서드를 통해서는 호출되지 않는다. 가령 `dd.get(k)`는 None을 반환한다.

## 3.4.2 `__missing__()` 메서드

`dict` 클래스를 상속하고 `__missing__()` 메서드를 정의하면 `dict.__getitem__()` 표준 메서드가 키를 발견할 수 없을 때 `KeyError`를 발생시키지 않고 `__missing__()` 메서드를 호출한다.

예를 들어, 키를 문자열로 변환하는 매핑을 생각해보자.

In [36]:
class StrKeyDict0(dict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key) # 키가 문자열인데 없으면 에러 발생시킴
        return self[str(key)]
        # 여기서 isinstance 가 없으면 __missing__() 메서드를 재귀적으로 호출함
    
    def get(self, key, default=None):
        try:
            return self[key]
            # 이렇게 시도했을 때 키가 없으면 __missing_()으로 넘어감
            # 그래서 그때마저 없으면 아래 except로 넘어감
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
    # 여기서 key in self 라고 쓰면 재귀적으로  __contains__() 가 호출됨

In [40]:
d = StrKeyDict0([('2','two'),('4','four')])

In [42]:
d['2']

'two'

In [43]:
d[4]

'four'

In [44]:
d[1]

KeyError: '1'

In [45]:
d.get('2')

'two'

In [46]:
d.get(4)

'four'

In [47]:
d.get(1,'N/A')

'N/A'

In [48]:
2 in d

True

In [49]:
1 in d

False

# 3.5 그 외 매핑형

### collections.OrderedDict

키를 삽입한 순서대로 유지함으로써 항목을 반복하는 순서를 예측할 수 있다. 따라서 `popitem()` 메서드를 사용할 수 있다.

### collections.ChainMap

매핑들의 목록을 담고 있으며 한꺼번에 모두 검색할 수 있다. 각 매핑을 차례대로 검색하고, 그중 하나에서라도 키가 검색되면 성공한다.

### collections.Counter

모든 키에 정수형 카운터를 갖고 있는 매핑. 기존 키를 갱신하면 카운터가 늘어난다. n개의 가장 널리 사용된 항목과 그들의 카운터로 구성된 튜플의 리스트를 반환하는 `most_commom([n])` 등의 메서드를 제공한다.

In [50]:
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [51]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 3})

In [52]:
ct.most_common(2)

[('a', 10), ('z', 3)]

# 3.6 UserDict 상속하기

`dict`보다는 `UserDict`를 상속해서 매핑형을 만드는 것이 쉽다. `UserDict`는 `dict`를 상속하지 않고 내부에 실제 항목을 담고 있는 `data`라고 하는 `dict` 객체를 갖고 있다. 이렇게 구현함으로써 `__setitem__()` 등의 특수 메서드를 구현할 때 발생하는 원치 않는 재귀적 호출을 피할 수 있으며, `__contains__()` 메서드를 간단히 구현할 수 있다.

In [53]:
class StrKeyDict(collections.UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [54]:
d = StrKeyDict([('2','two'),('4','four')])

In [55]:
d[2]

'two'

`UserDict` 클래스가 `MutableMapping`을 상속하므로 `StrKeyDict`는 매핑의 모든 기능을 가지게 된다. 따라서 `StrKeyDict0.get()`과 동일하게 구현된 `Mapping.get()`을 상속받으므로 구현할 필요가 없다.

# 3.7 불변 매핑

매핑형은 모두 가변형이지만, `types` 모듈의 `MappingProxyType` 래퍼 클래스를 이용하면 원래 매핑의 동적인 뷰를 제공하는 `mappingproxy` 객체를 반환받는다. 따라서 원래 매핑을 변경하면 `mappingproxy`에 반영되지만, `mappingproxy`를 직접 변경할 수는 없다.

In [56]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)

In [57]:
d_proxy

mappingproxy({1: 'A'})

In [58]:
d_proxy[1]

'A'

In [60]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [61]:
d[2] = 'x'

In [63]:
d_proxy

mappingproxy({1: 'A', 2: 'x'})

In [64]:
d_proxy[2]

'x'

# 3.8 집합 이론

집합 요소는 반드시 해시할 수 있어야 한다. `set`은 해시 가능하지 않지만 `frozenset`은 해시 가능하므로, `frozenset`이 `set`에 들어갈 수 있다. 또한 집합형은 다음과 같은 연산을 구현한다.

* a | b : 합집합
* a & b : 교집합
* a - b : 차집합

## 3.8.1 집합 리터럴

공집합은 리터럴로 표기할 수 없으므로, 반드시 `set()`으로 표기해야 한다. `{1,2,3}`과 같은 리터럴 집합 구문은 `set([1,2,3])`처럼 생성자를 호출하는 것보다 더 빠르고 가독성이 좋다(리터럴 구문을 사용하면 파이썬은 `BUILD_SET`이라는 특수 바이트코드를 실행한다.

In [65]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [66]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


`frozenset`에 대한 별도 리터럴 구문은 없다.

In [67]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## 3.8.2 지능형 집합

In [68]:
from unicodedata import name
{chr(i) for i in range(32,256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [74]:
name(chr(100))

'LATIN SMALL LETTER D'

In [81]:
name(chr(100),'') # 문자열의 영문 설명 출력

'LATIN SMALL LETTER D'

In [80]:
chr(240), name(chr(240))

('ð', 'LATIN SMALL LETTER ETH')

## 3.8.3 집합 연산

다른 건 별거 없고, `s ^ z` 는 $(s\cup z) - (s \cap z)$ 이다. 그리고 부분 집합 여부는 `s < z` 메서드를 사용한다. 또한 두 집합이 교집합이 없는지 보려면 `s.isdisjoint(z)` 메서드를 사용한다. 또한 두 집합이 아니라 여러 집합에 대한 연산을 수행할 때는 `a.update(b,c,d)` 등으로 사용하면 된다.

# 3.9 dict와 set의 내부 구조

어렵다...